In [1]:
%%capture
#Installing required Libraries wherever the code is being executed - run only if needed
!pip install soundex
!pip install jellyfish
!pip install pandasgui

In [2]:
#Importing required Libraries
import pandas as pd 
import jellyfish
import collections
import numpy as np
import soundex as sdx
#import fuzzy
#import pandasql as psql

In [3]:
# Please remember to navigate to "Shared with me" on your Gdrive home sceen
# Right click on the "HNY_Project" (the primary folder for this project) and select "Add shortcut to Drive"
# This way the below code can find a link to the Project dr folder through your own drive.

# Mounting your personal Gdrive to the Colab notebook
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [4]:
BK_name = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Standardization Output/Pre Final/BK_1850_name.csv')
MN_name = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Standardization Output/Pre Final/MN_1850_name.csv')

#BK_name = pd.read_csv('/gdrive/My Drive/Colab Notebooks/BK_1850_name.csv',low_memory=False)
#MN_name = pd.read_csv('/gdrive/My Drive/Colab Notebooks/MN_1850_name.csv',low_memory=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
BK_name = BK_name[['f_first_name', 'f_last_name', 'f_middle_name', 'f_title', 'junk_entries', 'f_refused_flag', 'f_race_info_avl', 'f_non_person_flag']]

MN_name = MN_name[['f_first_name', 'f_last_name', 'f_middle_name', 'f_title', 'junk_entries', 'f_refused_flag', 'f_race_info_avl', 'f_non_person_flag']]         

In [6]:
name_1850 = MN_name.append(BK_name)

In [11]:
name_1850_run = name_1850[['f_first_name', 'f_last_name', 'f_middle_name']]
name_1850_run.columns = ['first_name', 'last_name', 'middle_name']

# Phonetic Encoding of Names using different algorithms

In [ ]:
# name_sample = name_1850[['f_first_name', 'f_last_name', 'f_middle_name']].head(5000)
# name_sample.columns = ['first_name', 'last_name', 'middle_name']

In [7]:
def soundex_encoder(x):
  if pd.notnull(x):
    x = jellyfish.soundex(x)
  else:
    pass
  return x

def metaphone_encoder(x):
  if pd.notnull(x):
    x = jellyfish.metaphone(x)
  else:
    pass
  return x

In [8]:
def sx_generator(in_df):
  out_df = pd.DataFrame()

  for index, row in in_df.iterrows():
    try:
      ret_val = row.apply(soundex_encoder)
    except Exception:
      pass
    out_df = out_df.append(ret_val,ignore_index=True)  

  out_df.columns = ['first_name_sx', 'last_name_sx', 'middle_name_sx']

  return out_df

def metaphone_generator(in_df):
  out_df = pd.DataFrame()

  for index, row in in_df.iterrows():
    try:
      ret_val = row.apply(metaphone_encoder)
    except Exception:
      pass
    out_df = out_df.append(ret_val,ignore_index=True)  

  out_df.columns = ['first_name_mph', 'last_name_mph', 'middle_name_mph']

  return out_df

In [ ]:
#name_1850_enc = pd.merge(metaphone_generator(name_sample), sx_generator(name_sample), left_index=True, right_index=True)
#name_1850_enc = pd.merge(name_sample,name_1850_enc, left_index=True, right_index=True)

## Soundex Count & List Generation

In [12]:
name_1850_sx_enc = pd.merge(name_1850_run, sx_generator(name_1850_run), left_index=True, right_index=True)

In [14]:
name_1850_sx_enc['first_name_sx_count'] = name_1850_sx_enc.groupby('first_name_sx')['first_name_sx'].transform('count')
name_1850_sx_enc['middle_name_sx_count'] = name_1850_sx_enc.groupby('middle_name_sx')['middle_name_sx'].transform('count')
name_1850_sx_enc['last_name_sx_count'] = name_1850_sx_enc.groupby('last_name_sx')['last_name_sx'].transform('count')

In [15]:
# List of Names for the same enconding 
working_df = name_1850_sx_enc.groupby(['first_name_sx']).agg({'first_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['first_name_sx', 'first_name_sx_list']

name_1850_sx_enc = pd.merge(name_1850_sx_enc, 
                        working_df, 
                        left_on= 'first_name_sx', 
                        right_on='first_name_sx')

# List of Names for the same enconding 
working_df = name_1850_sx_enc.groupby(['middle_name_sx']).agg({'middle_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['middle_name_sx', 'middle_name_sx_list']

name_1850_sx_enc = pd.merge(name_1850_sx_enc, 
                        working_df, 
                        left_on= 'middle_name_sx', 
                        right_on='middle_name_sx')

# List of Names for the same enconding 
working_df = name_1850_sx_enc.groupby(['last_name_sx']).agg({'last_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['last_name_sx', 'last_name_sx_list']

name_1850_sx_enc = pd.merge(name_1850_sx_enc, 
                        working_df, 
                        left_on= 'last_name_sx', 
                        right_on='last_name_sx')

## Metaphone Count & List Generation

In [17]:
name_1850_mph_enc = pd.merge(name_1850_run, metaphone_generator(name_1850_run), left_index=True, right_index=True)

In [ ]:
name_1850_mph_enc['first_name_mph_count'] = name_1850_mph_enc.groupby('first_name_mph')['first_name_mph'].transform('count')
name_1850_mph_enc['middle_name_mph_count'] = name_1850_mph_enc.groupby('middle_name_mph')['middle_name_mph'].transform('count')
name_1850_mph_enc['last_name_mph_count'] = name_1850_mph_enc.groupby('last_name_mph')['last_name_mph'].transform('count')

In [ ]:
# List of Names for the same enconding 
working_df = name_1850_mph_enc.groupby(['first_name_mph']).agg({'first_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['first_name_mph', 'first_name_mph_list']

name_1850_mph_enc = pd.merge(name_1850_mph_enc, 
                        working_df, 
                        left_on= 'first_name_mph', 
                        right_on='first_name_mph')

# List of Names for the same enconding 
working_df = name_1850_mph_enc.groupby(['middle_name_mph']).agg({'middle_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['middle_name_mph', 'first_name_mph_list']

name_1850_mph_enc = pd.merge(name_1850_mph_enc, 
                        working_df, 
                        left_on= 'middle_name_mph', 
                        right_on='middle_name_mph')

# List of Names for the same enconding 
working_df = name_1850_mph_enc.groupby(['last_name_mph']).agg({'last_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['last_name_mph', 'first_name_mph_list']

name_1850_mph_enc = pd.merge(name_1850_mph_enc, 
                        working_df, 
                        left_on= 'last_name_mph', 
                        right_on='last_name_mph')

In [ ]:
sample = name_1850_sx_enc.sort_values(by='first_name_sx_count', ascending=False).head(40)
sample

,first_name,last_name,middle_name,first_name_sx,last_name_sx,middle_name_sx,first_name_sx_count,last_name_sx_count,first_name_sx_list,middle_name_sx_list,last_name_sx_list
39,JOHN,BACON,&,J500,B250,&000,603.0,38.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},"{besson, bacon}"
111,John,Barnes,clerk,J500,B652,C462,603.0,82.0,"{jane, johanna, joanna, jean, jona, jno, john,...","{clerk, clark}",{barnes}
37,Jahne,Arthur,&,J500,A636,&000,603.0,17.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},{arthur}
44,John,Bammann,&,J500,B550,&000,603.0,32.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},"{bammann, benham}"
46,John,Banker,&,J500,B526,&000,603.0,23.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},"{banker, benguerel, benkard}"
120,John,Ahx,shoemaker,J500,A200,S526,603.0,20.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{shoemaker},{ahx}
28,John,Anderson,&,J500,A536,&000,603.0,233.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},"{andrews, anderson, andreas, andreae}"
49,JOHN,BARCLAY,&,J500,B624,&000,603.0,37.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},"{barclay, birchall}"
40,John,Bacon,FE,J500,B250,F000,603.0,38.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{fe},"{besson, bacon}"
116,JOHN,BIGLEY,KE,J500,B240,K000,603.0,50.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{ke},"{bigley, bakewell, bigelow}"


# JSON Generation 

In [ ]:
sample = name_1850_sx_enc.groupby(['last_name_sx']).agg({'last_name': lambda x: set(x.str.lower())})
sample

,last_name
last_name_sx,
A126,{apgar}
A130,"{abbot, abbott}"
A141,{appleby}
A160,{avery}
A163,{aubert}
...,...
F200,{fick}
G626,{gregory}
H536,{hendrickson}


In [ ]:
print(sample.set_index('last_name_sx').rename(columns={'last_name':'index1'}).to_json())

KeyError: ignored

In [ ]:
list(sample.iloc[1,8])

['jane', 'johanna', 'joanna', 'jean', 'jona', 'jno', 'john', 'johann', 'jahne']

# Exploration

In [ ]:
name_1850_sx_enc.sort_values(by='first_name_sx_count', ascending=False).head(40)

,first_name,last_name,middle_name,first_name_sx,last_name_sx,middle_name_sx,first_name_sx_count,last_name_sx_count,first_name_sx_list,middle_name_sx_list,last_name_sx_list
39,JOHN,BACON,&,J500,B250,&000,603.0,38.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},"{besson, bacon}"
111,John,Barnes,clerk,J500,B652,C462,603.0,82.0,"{jane, johanna, joanna, jean, jona, jno, john,...","{clerk, clark}",{barnes}
37,Jahne,Arthur,&,J500,A636,&000,603.0,17.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},{arthur}
44,John,Bammann,&,J500,B550,&000,603.0,32.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},"{bammann, benham}"
46,John,Banker,&,J500,B526,&000,603.0,23.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},"{banker, benguerel, benkard}"
120,John,Ahx,shoemaker,J500,A200,S526,603.0,20.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{shoemaker},{ahx}
28,John,Anderson,&,J500,A536,&000,603.0,233.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},"{andrews, anderson, andreas, andreae}"
49,JOHN,BARCLAY,&,J500,B624,&000,603.0,37.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{&},"{barclay, birchall}"
40,John,Bacon,FE,J500,B250,F000,603.0,38.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{fe},"{besson, bacon}"
116,JOHN,BIGLEY,KE,J500,B240,K000,603.0,50.0,"{jane, johanna, joanna, jean, jona, jno, john,...",{ke},"{bigley, bakewell, bigelow}"


## Extras

In [ ]:
def df_soundex(df,in_col, out_col):
  hash_table = collections.defaultdict()
  for index, row in df.iterrows():
    try:
      if pd.isna(row[in_col]):
        hash_table[index] = float("NaN")
      else:
        hash_table[index] = jellyfish.soundex(row[in_col])
  
      df[out_col] = pd.Series(dict(hash_table),index=df.index)
    except Exception:
      pass
  return df

def df_metaphone(df,in_col, out_col):
  hash_table = collections.defaultdict()
  for index, row in df.iterrows():
      if pd.isna(row[in_col]):
        hash_table[index] = float("NaN")
      else:
        hash_table[index] = jellyfish.metaphone(row[in_col])
      
      df[out_col] = pd.Series(dict(hash_table),index=df.index)
  
  return df